In [5]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth
from folium import GeoJson, GeoJsonTooltip
from pygris import tracts
import branca
import os

data = pd.read_csv("Census_data_binned_v2.csv")

# Load tracts for Delaware
DE_tracts = tracts(state="DE", cb=True, cache=True)
DE_tracts['GEOID'] = DE_tracts['GEOID'].astype('int64')

# Merge with data
merged_df = pd.merge(DE_tracts, data, on='GEOID', how='left')
gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')

# Filter valid geometries
gdf = gdf[gdf.is_valid & (gdf.area > 0.0001)]

# Initialize the map
m = folium.Map(location=[38.9108, -75.5277], zoom_start=9, tiles="CartoDB positron")

Using the default year of 2021
Using FIPS code '10' for input 'DE'


C:\Users\RakshitMadan\AppData\Local\Temp\ipykernel_43348\2697610282.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = gdf[gdf.is_valid & (gdf.area > 0.0001)]


In [6]:
def create_map(data, gdf, metric, bin_column, file_name):
    """
    Create a Folium map for a specified metric and its bin range column.
    
    Parameters:
    - data: DataFrame containing the metric and bin range columns
    - gdf: GeoDataFrame for geographic information
    - metric: The column name of the metric to visualize
    - bin_column: The column name of the bin ranges for the legend
    - file_name: The file name to save the generated map (including path)
    """
    # Dynamically generate a color mapping from unique bins in the bin_column
    unique_bins = gdf[bin_column].dropna().unique()
    unique_bins.sort()  # Ensure bins are sorted for legend clarity

    # Define colors for the bins (YlOrRd scale with enough colors for bins)
    colors = [
        "#ffffcc", "#fed976", "#fd8d3c", "#f03b20", "#bd0026", "#800026"
    ]  # Extend this if more bins are present

    # Create a dynamic color mapping
    color_mapping = {bin_val: colors[i % len(colors)] for i, bin_val in enumerate(unique_bins)}

    def bin_color(feature):
        """Return the color based on the bin_column."""
        bin_value = feature['properties'].get(bin_column)
        return color_mapping.get(bin_value, 'gray')  # Default to gray if bin is missing

    # Initialize the map
    m = folium.Map(location=[38.9108, -75.5277], zoom_start=9, tiles="CartoDB positron")

    # Add GeoJson layer
    folium.GeoJson(
        gdf,
        style_function=lambda feature: {
            'fillColor': bin_color(feature),  # Use bin_color for color mapping
            'color': 'black',  # Boundary line color
            'weight': 0.5,  # Boundary line width
            'fillOpacity': 0.7,  # Transparency of fill color
        },
        tooltip=GeoJsonTooltip(
            fields=[metric, bin_column, "NAMELSAD", "NAMELSADCO", "STATE_NAME"],  # Add fields
            aliases=[
                f"{metric.replace('_', ' ').title()}:",
                f"{bin_column.replace('_', ' ').title()}:",
                "Tract Name:",
                "County Name:",
                "State Name:"
            ],
            localize=True
        )
    ).add_to(m)

    # Dynamically generate legend HTML with percentage signs
    legend_html = f"""
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        width: 200px;
        height: auto;
        background-color: white;
        border:2px solid grey;
        z-index:9999;
        font-size:14px;
        padding: 10px;
        ">
        <b>Legend</b><br>
    """
    for bin_val, color in color_mapping.items():
        legend_html += f"""
        <i style="background: {color}; width: 10px; height: 10px; display: inline-block;"></i> {bin_val}%<br>
    """
    legend_html += "</div>"

    # Add the legend to the map
    m.get_root().html.add_child(folium.Element(legend_html))

    # Save the map
    os.makedirs(os.path.dirname(file_name), exist_ok=True)  # Ensure the directory exists
    m.save(file_name)
    print(f"Map saved as: {file_name}")

In [8]:
# Metrics and their bin range columns
metrics = {
    "Pct_noHealthInsurance": "Pct_noHealthInsurance_bin_range",
    "Pct_internet": "Pct_internet_bin_range",
    "pct_rentBurdenend": "pct_rentBurdenend_bin_range",
    "Pct_transportAccess": "Pct_transportAccess_bin_range",
    "Pct_deviceAccess": "Pct_deviceAccess_bin_range",
    "Pct_minority": "Pct_minority_bin_range",
    "Pct_HsGrad": "Pct_HsGrad_bin_range",
    "Unemployment Rate":"Unemployment Rate_bin_range",
    "Pct_nonNative": "Pct_nonNative_bin_range",
    "Income Inequality (GINI Index)": "Income Inequality (GINI Index)_bin_range"
}

# Create maps for each metric and save to "Maps 3" folder
for metric, bin_column in metrics.items():
    file_name = f"Maps 3/{metric}_map.html".replace(" ", "_")  # Save files with clean names in Maps 3 folder
    create_map(data, gdf, metric, bin_column, file_name)

Map saved as: Maps_3/Pct_noHealthInsurance_map.html
Map saved as: Maps_3/Pct_internet_map.html
Map saved as: Maps_3/pct_rentBurdenend_map.html
Map saved as: Maps_3/Pct_transportAccess_map.html
Map saved as: Maps_3/Pct_deviceAccess_map.html
Map saved as: Maps_3/Pct_minority_map.html
Map saved as: Maps_3/Pct_HsGrad_map.html
Map saved as: Maps_3/Unemployment_Rate_map.html
Map saved as: Maps_3/Pct_nonNative_map.html
Map saved as: Maps_3/Income_Inequality_(GINI_Index)_map.html
